In [1]:
import os
import uuid
import shutil
#import pylatex
import pandas as pd
import lightkurve as lk 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from datetime import datetime
from tqdm.notebook import tqdm
from multiprocessing import cpu_count, Pool
#from pylatex.section import Chapter, Subsection
#from pylatex import Document, Section, Figure, NoEscape, Command

%run pipeline.ipynb

import warnings
warnings.filterwarnings("ignore")

In [2]:
bestlcs=pd.read_csv('makingtable/bestlcs.csv')
gaiatable=pd.read_csv('makingtable/cleanedgaiadata.csv')
id_table=pd.read_csv('makingtable/idswithdiscopaper.csv')

In [3]:
bestlclist=np.array([], dtype=object)

In [4]:
start_col=bestlcs.columns.get_loc('Sector 1')

In [5]:
for index, row in bestlcs.iterrows(): #iterate through each object
    for col in bestlcs.columns[start_col:]: #iterate through all the columns starting with the sectors
        value=row[col]
        if not pd.isna(value):
            bestlclist=np.append(bestlclist, [f'{row['pop_id']}_{value}'])

In [6]:
#now doing K2SFF lcs
K2SFFlist=np.loadtxt('makingtable/K2SFFlist.txt', dtype=str)

In [7]:
lclist=pd.concat([pd.DataFrame(bestlclist), pd.DataFrame(K2SFFlist)], ignore_index=True)
lclist[['pop_id', 'lcname']]=lclist[0].str.split('_', expand=True)
lclist.drop(columns=0, inplace=True)

In [8]:
# for curstring in lclist[:10]: #starting with only 10 for testing
#     parts=curstring.split("_")
#     pop_id=parts[0]
#     lcname=parts[1]
#     curlc=get_lc_string(lcname)
#     fig=multiplot3(curlc,lcname,gaiatable)

#     if not os.path.isdir(f'multiplots/{pop_id}/'):
#         os.mkdir(f'multiplots/{pop_id}/')
        
#     fig.savefig(f'multiplots/{pop_id}/{lcname}.png',bbox_inches='tight')

In [9]:
# for index, currow in lclist.iterrows(): 
#     pop_id=currow['pop_id']
#     lcname=currow['lcname']
    
#     if os.path.isfile(f'multiplots/{pop_id}/{lcname}.png'): #don't replot ones that already have been plotted
#         continue

#     curlc=get_lc_string(lcname)
#     try: #make sure that an error with plotting one doesn't break the loop
#         fig=multiplot3(curlc,lcname,gaiatable,id_table)
    
#         if not os.path.isdir(f'multiplots/{pop_id}/'): #check if folder exists and if not make it
#             os.mkdir(f'multiplots/{pop_id}/')
            
#         fig.savefig(f'multiplots/{pop_id}/{lcname}.png',bbox_inches='tight')
#     except Exception as e:
#         print(f'Error processing {lcname}: {e}')

In [10]:
# for curstring in K2SFFlist: 
#     parts=curstring.split("_")
#     pop_id=parts[0]
#     lcname=parts[1]
    
#     if os.path.isfile(f'multiplots/{pop_id}/{lcname}.png'): #don't replot ones that already have been plotted
#         continue

#     curlc=get_lc_string(lcname)
#     try: #make sure that an error with plotting one doesn't break the loop
#         fig=multiplot3(curlc,lcname,gaiatable,id_table)
    
#         if not os.path.isdir(f'multiplots/{pop_id}/'): #check if folder exists and if not make it
#             os.mkdir(f'multiplots/{pop_id}/')
            
#         fig.savefig(f'multiplots/{pop_id}/{lcname}.png',bbox_inches='tight')
#     except Exception as e:
#         print(f'Error processing {lcname}: {e}')

In [11]:
lclist

,pop_id,lcname
0,0,TIC-59129133-5-FFI-30min
1,0,TIC-59129133-32-FFI-10min
2,0,TIC-59129133-43-SPOC-120
3,0,TIC-59129133-44-SPOC-120
4,0,TIC-59129133-71-FFI-200s
...,...,...
978,200,EPIC-248131102-13-K2SFF-1800
979,208,EPIC-246776923-13-K2SFF-1800
980,209,EPIC-246938594-13-K2SFF-1800
981,210,EPIC-246969828-13-K2SFF-1800


In [12]:
for index, currow in lclist.iterrows():
    pop_id=currow['pop_id']
    lcname=currow['lcname']

    idrow=id_table[id_table.pop_id.astype('str')==pop_id] #first lookup idtable row

    #if not (idrow.disco_paper.values[0]=='Stauffer_2017' or idrow.disco_paper.values[0]=='Stauffer_2018'): #skip the ones we aren't plotting
        #continue

    per1=idrow.per.values[0]
    per2=idrow.per2.values[0]


    if not os.path.isdir(f'multiplots/{pop_id}/'): #check if folder exists and if not make it
        os.mkdir(f'multiplots/{pop_id}/')

    #plot ones with found period
    if os.path.isfile(f'multiplots/{pop_id}/{lcname}.png'): #don't replot ones that already have been plotted
        continue

    curlc=get_lc_string(lcname)

    try: #make sure that an error with plotting one doesn't break the loop
        fig=multiplot3(curlc,lcname,gaiatable,id_table) 
        fig.savefig(f'multiplots/{pop_id}/{lcname}.png',bbox_inches='tight')
    except Exception as e:
        print(f'Error processing {lcname}: {e}')
    
    #plot first period
    if os.path.isfile(f'multiplots/{pop_id}/{lcname}_{per1}.png'): #don't replot ones that already have been plotted
        continue

    try: #make sure that an error with plotting one doesn't break the loop
        fig=multiplot3(curlc,lcname,gaiatable,id_table,period=per1) 
        fig.savefig(f'multiplots/{pop_id}/{lcname}_{per1}.png',bbox_inches='tight')
    except Exception as e:
        print(f'Error processing {lcname}: {e}')

    if not pd.isna(per2): #plot second period if applicable
        if os.path.isfile(f'multiplots/{pop_id}/{lcname}_{per2}.png'): #don't replot ones that already have been plotted
            continue
    
        try: #make sure that an error with plotting one doesn't break the loop
            fig=multiplot3(curlc,lcname,gaiatable,id_table,period=per2) 
            fig.savefig(f'multiplots/{pop_id}/{lcname}_{per2}.png',bbox_inches='tight')
        except Exception as e:
            print(f'Error processing {lcname}: {e}')

Error processing TIC-404335106-38-FFI-10min: index -1 is out of bounds for axis 0 with size 0
Error processing TIC-404335106-38-FFI-10min: index 0 is out of bounds for axis 0 with size 0
Error processing TIC-300651846-13-FFI-30min: index -1 is out of bounds for axis 0 with size 0
Error processing TIC-300651846-13-FFI-30min: index 0 is out of bounds for axis 0 with size 0
Error processing TIC-442571495-91-FFI-200s: index -1 is out of bounds for axis 0 with size 0
Error processing TIC-442571495-91-FFI-200s: index 0 is out of bounds for axis 0 with size 0
Error processing TIC-442571495-91-FFI-200s: index 0 is out of bounds for axis 0 with size 0
Error processing TIC-442575691-91-FFI-200s: index -1 is out of bounds for axis 0 with size 0
Error processing TIC-442575691-91-FFI-200s: index 0 is out of bounds for axis 0 with size 0
Error processing TIC-442575691-91-FFI-200s: index 0 is out of bounds for axis 0 with size 0
Error processing TIC-244161191-42-FFI-10min: index -1 is out of bounds f